In [25]:
import os
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from sentiment_analyzer import get_sentiment_dictionary_from_csv_path


data_path = "C:/Users/Asus/Desktop/ttds-proj/backend/data/"
outlet_folders = ["bbc", "gbn", "ind", "tele"]

all_file_paths = [
    os.listdir(data_path + outlet_folders[outlet_index])
    for outlet_index in range(len(outlet_folders))
]

csv_dataframe = pd.read_csv(data_path + outlet_folders[0] + "/" + all_file_paths[0][0])

content_series = csv_dataframe['content']
doc_id_series = csv_dataframe['doc_id']

In [8]:
len(all_file_paths[3])

11

A function should take in a .csv path, model and tokenizer; then give back a tuple of its document id integer and sentiment triple.

In [ ]:
csv_sentiment_dictionary = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

csv_path = data_path + outlet_folders[0] + "/" + all_file_paths[0][0]

csv_sentiment_dictionary = analyze_sentiment_from_csv_path(
    csv_path, device, model, tokenizer, csv_sentiment_dictionary
)



In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
# model_name = "fhamborg/roberta-targeted-sentiment-classification-newsarticles"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.to(device)

text = content_series[1]


t_0 = time.time()
# Encode the text
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(
    device
)
t_1 = time.time()
print(f"Time taken for tokenizing: {t_1 - t_0:.4f} seconds")


t_0 = time.time()
# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
# Process the output
sentiments = (
    torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu")[0].float().tolist()
)  # first is negative, second neutral, third positive
t_1 = time.time()
print(f"Time taken for inference: {t_1 - t_0:.4f} seconds")

print(sentiments)

Using device: cuda
Time taken for tokenizing: 0.0040 seconds
Time taken for inference: 0.0220 seconds
[0.0003374790831003338, 0.005182734690606594, 0.994479775428772]


In [18]:
model.to(device)

csv_sentiment_dictionary = {}

for index, text in enumerate(tqdm(content_series)):
    try:
        inputs = tokenizer(
            text, return_tensors="pt", truncation=True, max_length=512
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            sentiments = torch.nn.functional.softmax(outputs.logits, dim=-1).to("cpu")[0].float().numpy()
        # Round the first 2 elements
        rounded_sentiments = np.round(sentiments[:-1], decimals=2)
        # Ensure the sum equals 1 by adjusting the last element
        last_element = 1 - np.sum(rounded_sentiments)
        rounded_sentiments = list(rounded_sentiments) + [np.round(last_element, decimals=2)]
        csv_sentiment_dictionary[doc_id_series[index]] = rounded_sentiments
    except:
        csv_sentiment_dictionary[doc_id_series[index]] = None

  0%|          | 0/504 [00:00<?, ?it/s]

In [13]:
csv_sentiment_dictionary

{0: [0.96, 0.01, 0.03],
 1: [0.0, 0.01, 0.99],
 2: [0.16, 0.01, 0.83],
 3: [0.0, 0.99, 0.01],
 4: [0.0, 0.0, 1.0],
 5: [0.49, 0.51, 0.0],
 6: [0.53, 0.47, 0.0],
 7: [1.0, 0.0, 0.0],
 8: [0.82, 0.01, 0.17],
 9: [0.0, 0.0, 1.0],
 10: [0.0, 0.07, 0.93],
 11: [0.0, 1.0, 0.0],
 12: [0.01, 0.01, 0.98],
 13: [0.0, 1.0, 0.0],
 14: [0.0, 0.99, 0.01],
 15: [0.0, 0.95, 0.05],
 16: [0.0, 1.0, 0.0],
 17: [0.0, 0.02, 0.98],
 18: [0.0, 0.97, 0.03],
 19: [0.0, 0.0, 1.0],
 20: [0.0, 0.0, 1.0],
 21: [0.0, 0.93, 0.07],
 22: [0.0, 1.0, 0.0],
 23: [0.0, 1.0, 0.0],
 24: [0.0, 0.41, 0.59],
 25: [0.0, 1.0, 0.0],
 26: [0.0, 1.0, 0.0],
 27: [0.07, 0.93, 0.0],
 28: [0.0, 1.0, 0.0],
 29: [0.68, 0.05, 0.27],
 30: [0.0, 1.0, 0.0],
 31: [0.0, 0.12, 0.88],
 32: [0.02, 0.77, 0.21],
 33: [0.0, 0.99, 0.01],
 34: [0.0, 1.0, 0.0],
 35: [0.99, 0.0, 0.01],
 36: [0.04, 0.96, 0.0],
 37: [0.09, 0.27, 0.64],
 38: [0.0, 0.99, 0.01],
 39: [0.0, 1.0, 0.0],
 40: [0.24, 0.76, 0.0],
 41: [0.03, 0.0, 0.97],
 42: [0.0, 1.0, 0.0],
 43: 